# How to Choose Your Hotel?

## Introduction Problem

Graduation is the dream of all parents to their children who have completed their studies. Sometimes many parents come all the way to attend the graduation process of their child. **Usually those from out of town will stay at the closest hotel to the university before coming to the graduation ceremony so as not to be late and save time. Interesting places around the hotel can also attract people to stay**. 

To determine the recommended hotel, we can use the clustering technique to divide the nearest hotel group into several segments or groups. The data used is the hotel data closest to the Indonesian Computer University (UNIKOM) and the places around the hotel which were taken using the FourSquare API. Furthermore, they will be grouped into several segments using the K-means algorithm to determine the right hotel choice.

## Data

Based on definition of our problem, we need data about:
- Hotels around the Indonesian Computer University with radius 500 meter.
- Places near the hotel with coordinate location. 

The data is retrieved using the **Fousquare API**

## Methodology

### Import Necessary Library

In [1]:
# load necessary library
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors
import plotly.express as px
from sklearn.cluster import KMeans

!pip install folium #uncomment if you already installed folium
import folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.4 MB/s  eta 0:00:01


### Get access to Foursquare API

In [ ]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare Secret
ACCESS_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare access token
VERSION = '20210101' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Define Location

In [3]:
address = 'Universitas Komputer Indonesia, Jawa Barat, Indonesia'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-6.8868408 107.6151136


### 1. Search Hotel around with radius 500m

In [4]:
search_key = 'hotel'
radius = 500
url_key = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude,
    ACCESS_TOKEN, VERSION, search_key, radius, LIMIT)
# get hotels data in json file format
results = requests.get(url_key).json()
hotels = results['response']['venues']

# tranform json file into a dataframe
df_hotel = pd.json_normalize(hotels)
df_hotel.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet
0,4c72303f9e706dcbf3374a62,Hotel Neo Dipatiukur,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619135482,False,Jl. Dipatiukur No. 72-74,-6.889919,107.616451,"[{'label': 'display', 'lat': -6.88991914858835...",373,40132,ID,Bandung,Jawa Barat,Indonesia,"[Jl. Dipatiukur No. 72-74, Bandung, Jawa Barat...",133925097,NaN
1,515e6be5e4b0c134437ee63a,Patra Jasa Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619135482,False,Jl. Ir. H. Juanda No.132,-6.889154,107.614085,"[{'label': 'display', 'lat': -6.88915357376354...",281,40132,ID,Bandung,Jawa Barat,Indonesia,[Jl. Ir. H. Juanda No.132 (Jl. Ir. H. Juanda N...,NaN,Jl. Ir. H. Juanda No. 132
2,4bb73d162f70c9b68e668630,Scarlet Dago Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619135482,False,Jalan Siliwangi No. 5,-6.885105,107.612117,"[{'label': 'display', 'lat': -6.88510543232216...",383,40131,ID,Bandung,Jawa Barat,Indonesia,"[Jalan Siliwangi No. 5 (Dago), Bandung, Jawa B...",NaN,Dago
3,4be81e5dd837c9b6efe4a406,Royal Dago Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619135482,False,Jl. Ir. H. Djuanda No. 156-169,-6.886584,107.613328,"[{'label': 'display', 'lat': -6.88658352860527...",199,NaN,ID,Bandung,Jawa Barat,Indonesia,"[Jl. Ir. H. Djuanda No. 156-169, Bandung, Jawa...",NaN,NaN
4,4dfef268483b8f033b425a03,Hotel Puspa Kencana,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1619135482,False,Jl. Siliwangi no. 10,-6.884576,107.612229,"[{'label': 'display', 'lat': -6.8845756140164,...",406,NaN,ID,Bandung,Jawa Barat,Indonesia,"[Jl. Siliwangi no. 10, Bandung, Jawa Barat]",NaN,NaN


#### Cleaning Hotels Data

In [52]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['id','name'] + [col for col in df_hotel.columns if col.startswith('location.')]
hotel_filtered = df_hotel.loc[:, filtered_columns]

# clean column names by keeping only last term
hotel_filtered.columns = [column.split('.')[-1] for column in hotel_filtered.columns]
df_hotel_filtered = hotel_filtered[['id', 'name', 'address', 'lat', 'lng']]

df_hotel_filtered.head()

,id,name,address,lat,lng
0,4c72303f9e706dcbf3374a62,Hotel Neo Dipatiukur,Jl. Dipatiukur No. 72-74,-6.889919,107.616451
1,515e6be5e4b0c134437ee63a,Patra Jasa Hotel,Jl. Ir. H. Juanda No.132,-6.889154,107.614085
2,4bb73d162f70c9b68e668630,Scarlet Dago Hotel,Jalan Siliwangi No. 5,-6.885105,107.612117
3,4be81e5dd837c9b6efe4a406,Royal Dago Hotel,Jl. Ir. H. Djuanda No. 156-169,-6.886584,107.613328
4,4dfef268483b8f033b425a03,Hotel Puspa Kencana,Jl. Siliwangi no. 10,-6.884576,107.612229


In [53]:
# Filtering name
df_hotel_filtered['name'] = df_hotel_filtered['name'].apply(lambda x : x.title().replace('Bandung', '').replace('Dago','').strip())
df_hotel_filtered['name']

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0                                  Hotel Neo Dipatiukur
1                                      Patra Jasa Hotel
2                                        Scarlet  Hotel
3                                          Royal  Hotel
4                                   Hotel Puspa Kencana
5                                      Patra Jasa Hotel
6                                 Geulis Boutique Hotel
7                                       Hotel Pratajasa
8                                        Hotel Sheraton
9                                        Hotel Sheraton
10                               Beehive Boutique Hotel
11                                         Hotel Royale
12                                     Hotel Patra Jasa
13                                     Hotel Patra Jasa
14                                         Hotel Akasia
15                                   Grand Unikom Hotel
16                                  Hotel Safari Garden
17                                     Hotel Pat

In [73]:
# choosing the hotels by index
name_index = [0,3,4,6,8,11,13,14,16,19,20,23,24,25,
              27,32,35,37,38,39,40,44,45,46,48]
df_hotel_filtered = df_hotel_filtered.loc[name_index].drop_duplicates(subset='name', keep='last')
df_hotel_filtered.sort_index(inplace=True)
df_hotel_filtered.head()

,id,name,address,lat,lng
0,4c72303f9e706dcbf3374a62,Hotel Neo Dipatiukur,Jl. Dipatiukur No. 72-74,-6.889919,107.616451
3,4be81e5dd837c9b6efe4a406,Royal Hotel,Jl. Ir. H. Djuanda No. 156-169,-6.886584,107.613328
4,4dfef268483b8f033b425a03,Hotel Puspa Kencana,Jl. Siliwangi no. 10,-6.884576,107.612229
6,4bef544ff2712d7f21a9fbd8,Geulis Boutique Hotel,Jl. Ir. H Juanda,-6.891400,107.612839
8,5479cfc2498e1e48fdb9b17c,Hotel Sheraton,NaN,-6.885765,107.614036


#### Create Hotel Map

In [74]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the UNIKOM

# add a red circle marker to represent the UNIKOM
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Unikom',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)

# add the hotels as blue circle markers
for lat, lng, name in zip(df_hotel_filtered.lat, df_hotel_filtered.lng, df_hotel_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=name,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map

### 2. Explore Places near the hotels

In [56]:
# function to add places near the hotel
def get_nearby_hotels(latitudes, longitudes, radius=500):
    url_nearby = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit=30'.format(
        CLIENT_ID, CLIENT_SECRET, latitudes, longitudes, VERSION, radius)
    
    res_nearby = requests.get(url_nearby).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    df_nearby = pd.DataFrame([(v['venue']['name'], 
                               v['venue']['location']['lat'],
                               v['venue']['location']['lng'],
                               v['venue']['categories'][0]['name']) for v in res_nearby])
    df_nearby.columns = ['Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

    return df_nearby

In [75]:
# get list nearby places each hotel
list_nearby = []
for hotel, lat, lng, in zip(df_hotel_filtered.name, df_hotel_filtered.lat, df_hotel_filtered.lng):
    nearby_hotels = get_nearby_hotels(lat, lng)
    nearby_hotels['Hotel Name'] = hotel
    nearby_hotels['Hotel Latitude'] = lat
    nearby_hotels['Hotel Longitude'] = lng
    list_nearby.append(nearby_hotels)
       
df_nearby_hotels = pd.concat(list_nearby).reset_index(drop=True)
df_nearby_hotels.head()

,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Hotel Name,Hotel Latitude,Hotel Longitude
0,Hotel Neo Dipatiukur,-6.889919,107.616451,Hotel,Hotel Neo Dipatiukur,-6.889919,107.616451
1,Koffie Opa,-6.891955,107.616803,Coffee Shop,Hotel Neo Dipatiukur,-6.889919,107.616451
2,Mie Merapi,-6.891459,107.616998,Noodle House,Hotel Neo Dipatiukur,-6.889919,107.616451
3,Hotel Patra Jasa Bandung,-6.889111,107.614090,Hotel,Hotel Neo Dipatiukur,-6.889919,107.616451
4,Bebeke Om Aris,-6.892662,107.617820,Asian Restaurant,Hotel Neo Dipatiukur,-6.889919,107.616451


In [76]:
print('Total Categories is', df_nearby_hotels['Venue Category'].nunique())

Total Categories is 44


In [77]:
# top 10 nearby places
nearby_categories = df_nearby_hotels['Venue Category'].value_counts()[:10].reset_index()
nearby_categories.columns = ['Venue', 'Total Venue']
fig = px.bar(nearby_categories, x='Venue', y='Total Venue', color='Venue')
fig.show()

#### One Hot Encoding By Venue Category

In [78]:
hotels_grouped.shape

(26, 44)

In [79]:
# remove hotel in nearby places
df_nearby_hotels = df_nearby_hotels[df_nearby_hotels['Venue Category'] != 'Hotel']

# one hot encoding by category
hotels_onehot = pd.get_dummies(df_nearby_hotels[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
hotels_onehot['Hotel Name'] = df_nearby_hotels['Hotel Name'] 

# move neighborhood column to the first column
fixed_columns = [hotels_onehot.columns[-1]] + list(hotels_onehot.columns[:-1])
hotels_onehot = hotels_onehot[fixed_columns]

# grouping by hotel name
hotels_grouped = hotels_onehot.groupby('Hotel Name').mean().reset_index()
hotels_grouped.head()

,Hotel Name,Acehnese Restaurant,American Restaurant,Arcade,Asian Restaurant,Bakery,Bistro,Breakfast Spot,Bridge,Burger Joint,Café,Chinese Restaurant,Cocktail Bar,Coffee Shop,College Auditorium,College Gym,College Stadium,Convenience Store,Dessert Shop,Diner,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Garden,Gas Station,Ice Cream Shop,Indonesian Restaurant,Juice Bar,Kids Store,Lounge,Music Venue,Noodle House,Office,Park,Pizza Place,Salon / Barbershop,Seafood Restaurant,Shoe Store,Spa,Sporting Goods Shop,Steakhouse,Sundanese Restaurant,Toy / Game Store
0,Crowne Hotel,0.000000,0.0,0.035714,0.035714,0.00,0.035714,0.000000,0.0,0.035714,0.178571,0.0,0.071429,0.142857,0.0,0.0,0.0,0.071429,0.071429,0.0,0.035714,0.000000,0.000000,0.0,0.0,0.0,0.035714,0.071429,0.0,0.00,0.035714,0.0,0.107143,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.035714,0.000000,0.000000,0.00
1,Fitness Center Galeri Ciumbuleuit Hotel & Apar...,0.035714,0.0,0.035714,0.035714,0.00,0.000000,0.035714,0.0,0.035714,0.178571,0.0,0.071429,0.142857,0.0,0.0,0.0,0.071429,0.071429,0.0,0.035714,0.035714,0.000000,0.0,0.0,0.0,0.000000,0.071429,0.0,0.00,0.035714,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.035714,0.000000,0.00
2,Geulis Boutique Hotel,0.000000,0.0,0.120000,0.040000,0.04,0.000000,0.000000,0.0,0.000000,0.160000,0.0,0.000000,0.120000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.040000,0.120000,0.0,0.04,0.040000,0.0,0.080000,0.0,0.0,0.0,0.0,0.0,0.08,0.04,0.040000,0.000000,0.000000,0.04
3,Grand Ball Room Panghegar Hotel,0.000000,0.0,0.000000,0.074074,0.00,0.000000,0.000000,0.0,0.037037,0.185185,0.0,0.074074,0.111111,0.0,0.0,0.0,0.074074,0.037037,0.0,0.074074,0.000000,0.037037,0.0,0.0,0.0,0.037037,0.074074,0.0,0.00,0.037037,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.037037,0.00
4,Grand Kopo Hotel,0.035714,0.0,0.000000,0.071429,0.00,0.000000,0.000000,0.0,0.035714,0.178571,0.0,0.035714,0.142857,0.0,0.0,0.0,0.071429,0.071429,0.0,0.035714,0.035714,0.035714,0.0,0.0,0.0,0.000000,0.107143,0.0,0.00,0.035714,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.035714,0.000000,0.00


#### Get Top 10 Places Near Each The Hotel

In [80]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Hotel Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
nearby_hotels_sorted = pd.DataFrame(columns=columns)
nearby_hotels_sorted['Hotel Name'] = hotels_grouped['Hotel Name']

for ind in np.arange(len(hotels_grouped)):
    nearby_hotels_sorted.iloc[ind, 1:] = return_most_common_venues(hotels_grouped.iloc[ind, :], num_top_venues)

nearby_hotels_sorted.head()

,Hotel Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Crowne Hotel,Café,Coffee Shop,Noodle House,Convenience Store,Indonesian Restaurant,Cocktail Bar,Dessert Shop,Burger Joint,Ice Cream Shop,Lounge
1,Fitness Center Galeri Ciumbuleuit Hotel & Apar...,Café,Coffee Shop,Convenience Store,Indonesian Restaurant,Cocktail Bar,Dessert Shop,Noodle House,Burger Joint,Food Court,Lounge
2,Geulis Boutique Hotel,Café,Indonesian Restaurant,Arcade,Coffee Shop,Shoe Store,Noodle House,Toy / Game Store,Kids Store,Lounge,Ice Cream Shop
3,Grand Ball Room Panghegar Hotel,Café,Coffee Shop,Noodle House,Fast Food Restaurant,Convenience Store,Asian Restaurant,Indonesian Restaurant,Cocktail Bar,Dessert Shop,Sundanese Restaurant
4,Grand Kopo Hotel,Café,Coffee Shop,Indonesian Restaurant,Asian Restaurant,Convenience Store,Dessert Shop,Noodle House,Food Truck,Lounge,Burger Joint


In [81]:
nearby_hotels_sorted.shape

(25, 11)

## 3. Hotel Segmentation

In [82]:
hotels_grouped_clustering = hotels_grouped.drop('Hotel Name', axis=1)

# run k-means clustering
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hotels_grouped_clustering)
# add clustering labels
nearby_hotels_sorted['Cluster'] = kmeans.labels_

hotel_data = df_nearby_hotels[['Hotel Name', 'Hotel Latitude', 'Hotel Longitude']]

# merge hotel data with hotels_grouped
hotels_merged = pd.merge(hotel_data, nearby_hotels_sorted, on='Hotel Name')
hotels_merged = hotels_merged.drop_duplicates(subset=['Hotel Name'], keep='last').reset_index(drop=True)
hotels_merged.head()

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,Hotel Neo Dipatiukur,-6.889919,107.616451,Café,Coffee Shop,Dessert Shop,Arcade,Noodle House,Breakfast Spot,Indonesian Restaurant,Lounge,Toy / Game Store,Bakery,1
1,Royal Hotel,-6.886584,107.613328,Café,Indonesian Restaurant,Coffee Shop,Arcade,Asian Restaurant,Lounge,Cocktail Bar,Convenience Store,Bridge,Noodle House,3
2,Hotel Puspa Kencana,-6.884576,107.612229,Café,Coffee Shop,Cocktail Bar,Ice Cream Shop,Indonesian Restaurant,Lounge,Bridge,Noodle House,Fast Food Restaurant,College Gym,3
3,Geulis Boutique Hotel,-6.891400,107.612839,Café,Indonesian Restaurant,Arcade,Coffee Shop,Shoe Store,Noodle House,Toy / Game Store,Kids Store,Lounge,Ice Cream Shop,1
4,Hotel Sheraton,-6.885765,107.614036,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Bridge,Noodle House,Lounge,3


#### Create Cluster Map

In [83]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(hotels_merged['Hotel Latitude'], hotels_merged['Hotel Longitude'], hotels_merged['Hotel Name'], hotels_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [84]:
hotels_merged.Cluster.value_counts()

3    11
0     6
1     4
2     3
4     1
Name: Cluster, dtype: int64

#### Recommended Hotel

In [85]:
# cluster 0
hotels_merged[hotels_merged['Cluster'] == 0]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
6,Hotel Patra Jasa,-6.889111,107.614090,Coffee Shop,Café,Noodle House,Cocktail Bar,Indonesian Restaurant,Bakery,Lounge,Fast Food Restaurant,Sporting Goods Shop,Arcade,0
7,Hotel Akasia,-6.886620,107.617260,Café,Coffee Shop,Indonesian Restaurant,Noodle House,Fast Food Restaurant,Ice Cream Shop,Dessert Shop,Convenience Store,Asian Restaurant,Lounge,0
14,Crowne Hotel,-6.888255,107.615798,Café,Coffee Shop,Noodle House,Convenience Store,Indonesian Restaurant,Cocktail Bar,Dessert Shop,Burger Joint,Ice Cream Shop,Lounge,0
15,Grand Ball Room Panghegar Hotel,-6.887248,107.616684,Café,Coffee Shop,Noodle House,Fast Food Restaurant,Convenience Store,Asian Restaurant,Indonesian Restaurant,Cocktail Bar,Dessert Shop,Sundanese Restaurant,0
16,Fitness Center Galeri Ciumbuleuit Hotel & Apar...,-6.888450,107.616193,Café,Coffee Shop,Convenience Store,Indonesian Restaurant,Cocktail Bar,Dessert Shop,Noodle House,Burger Joint,Food Court,Lounge,0
22,Grand Kopo Hotel,-6.888047,107.616955,Café,Coffee Shop,Indonesian Restaurant,Asian Restaurant,Convenience Store,Dessert Shop,Noodle House,Food Truck,Lounge,Burger Joint,0


In [86]:
# cluster 1
hotels_merged[hotels_merged['Cluster'] == 1]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
0,Hotel Neo Dipatiukur,-6.889919,107.616451,Café,Coffee Shop,Dessert Shop,Arcade,Noodle House,Breakfast Spot,Indonesian Restaurant,Lounge,Toy / Game Store,Bakery,1
3,Geulis Boutique Hotel,-6.891400,107.612839,Café,Indonesian Restaurant,Arcade,Coffee Shop,Shoe Store,Noodle House,Toy / Game Store,Kids Store,Lounge,Ice Cream Shop,1
18,Hotel Arimbi Stasion,-6.890589,107.616992,Coffee Shop,Indonesian Restaurant,Café,Arcade,Asian Restaurant,Noodle House,Breakfast Spot,Kids Store,Dessert Shop,Toy / Game Store,1
23,Hotel Puri Gardenia,-6.890862,107.616674,Coffee Shop,Arcade,Café,Noodle House,Breakfast Spot,Ice Cream Shop,Indonesian Restaurant,Kids Store,Dessert Shop,Toy / Game Store,1


In [87]:
# cluster 2
hotels_merged[hotels_merged['Cluster'] == 2]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
9,Hotel Efita,-6.886307,107.617765,Asian Restaurant,Café,Indonesian Restaurant,Coffee Shop,Convenience Store,Dessert Shop,Fast Food Restaurant,Noodle House,Ice Cream Shop,Food Truck,2
21,Hotel Lotus,-6.885058,107.619329,Convenience Store,Asian Restaurant,Coffee Shop,Dessert Shop,Indonesian Restaurant,Food Truck,Juice Bar,Bistro,Burger Joint,Café,2
24,"Hotel Alila Villas Soori, Tabanan ,Bali",-6.883052,107.618287,Convenience Store,Ice Cream Shop,Asian Restaurant,Dessert Shop,Sundanese Restaurant,Coffee Shop,Indonesian Restaurant,Chinese Restaurant,Café,Burger Joint,2


In [88]:
# cluster 3
hotels_merged[hotels_merged['Cluster'] == 3]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
1,Royal Hotel,-6.886584,107.613328,Café,Indonesian Restaurant,Coffee Shop,Arcade,Asian Restaurant,Lounge,Cocktail Bar,Convenience Store,Bridge,Noodle House,3
2,Hotel Puspa Kencana,-6.884576,107.612229,Café,Coffee Shop,Cocktail Bar,Ice Cream Shop,Indonesian Restaurant,Lounge,Bridge,Noodle House,Fast Food Restaurant,College Gym,3
4,Hotel Sheraton,-6.885765,107.614036,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Bridge,Noodle House,Lounge,3
5,Hotel Royale,-6.884864,107.614899,Café,Cocktail Bar,Coffee Shop,Ice Cream Shop,Indonesian Restaurant,Asian Restaurant,Burger Joint,Lounge,Convenience Store,Noodle House,3
8,Hotel Safari Garden,-6.887974,107.612964,Café,Fast Food Restaurant,Coffee Shop,Indonesian Restaurant,Cocktail Bar,Bridge,College Stadium,Gas Station,Ice Cream Shop,Lounge,3
10,Hotel Shangri-La,-6.884710,107.613377,Café,Coffee Shop,Cocktail Bar,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Lounge,Bridge,Noodle House,Fast Food Restaurant,3
11,Prime Park Hotel,-6.887748,107.613901,Café,Fast Food Restaurant,Coffee Shop,Indonesian Restaurant,Cocktail Bar,Bridge,College Stadium,Gas Station,Ice Cream Shop,Lounge,3
12,Proyek Hotel Lor In Lembang,-6.885684,107.614102,Café,Fast Food Restaurant,Coffee Shop,Cocktail Bar,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Bridge,Noodle House,Lounge,3
13,Wirton Hotel,-6.886820,107.613599,Café,Asian Restaurant,Coffee Shop,Indonesian Restaurant,Cocktail Bar,Fast Food Restaurant,Arcade,Ice Cream Shop,Convenience Store,Park,3
19,Sheo Resort Hotel,-6.885247,107.613813,Café,Coffee Shop,Cocktail Bar,Asian Restaurant,Ice Cream Shop,Indonesian Restaurant,Lounge,Bridge,Noodle House,Fast Food Restaurant,3


In [89]:
# cluster 4
hotels_merged[hotels_merged['Cluster'] == 4]

,Hotel Name,Hotel Latitude,Hotel Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
17,Hotel Cemerlang,-6.888986,107.618706,Asian Restaurant,Coffee Shop,Noodle House,Convenience Store,Indonesian Restaurant,Café,Burger Joint,Dessert Shop,Food Truck,Breakfast Spot,4
